# Word Embedding - Home Assigment
## Dr. Omri Allouche 2018. YData Deep Learning Course

[Open in Google Colab](https://colab.research.google.com/github/omriallouche/deep_learning_course/blob/master/DL_word_embedding_assignment.ipynb)
    
    
In this exercise, you'll use word vectors trained on a corpus of 380,000 lyrics of songs from MetroLyrics (https://www.kaggle.com/gyani95/380000-lyrics-from-metrolyrics).  
The dataset contains these fields for each song, in CSV format:
1. index
1. song
1. year
1. artist
1. genre
1. lyrics

Before doing this exercise, we recommend that you go over the "Bag of words meets bag of popcorn" tutorial (https://www.kaggle.com/c/word2vec-nlp-tutorial)

Other recommended resources:
- https://rare-technologies.com/word2vec-tutorial/
- https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial

### Train word vectors
Train word vectors using the Skipgram Word2vec algorithm and the gensim package.
Make sure you perform the following:
- Tokenize words
- Lowercase all words
- Remove punctuation marks
- Remove rare words
- Remove stopwords

Use 300 as the dimension of the word vectors. Try different context sizes.

In [0]:
!pip install livelossplot


In [33]:
import pandas as pd
import string
import nltk 
from gensim.models import Word2Vec
import re
import numpy as np
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
pd.set_option('display.width', -1)
pd.set_option('max_colwidth', -1)
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier,XGBRegressor
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
stop_words = set(stopwords.words("english"))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
lyrics_df = pd.read_csv("/content/drive/My Drive/lyrics.csv", usecols=['genre', 'lyrics'])
lyrics_df.sample(2)

,genre,lyrics
190771,Other,NaN
271327,Rock,"Throw me in the water\n'Cause you know that I could never swim\nI'll think of you as I go down\nAnd you put yourself in order\n'Cause you know that I could never count\nAs high as you my nose would bleed\nAnd your heart is like a stone\nYour flesh has turned to mold\nDon't feel bad if hell is not the place you hoped it'll be\nAnd you put aside your dagger\n'Cause you know that you might need it soon\nTo cut me open, feel around inside\nAnd you make like I'm your father\nSo the blood that spills is all the same\nTo me and everyone you know\nYour heart is like a stone\nAnd your flesh has turned to mold\nDon't feel bad if hell is not the place you hoped it'll be\nAnd you say it doesn't matter\n'Cause you know from all those books you never read\nYou knew how they would end"


In [0]:
lyrics_df = lyrics_df.where((pd.notnull(lyrics_df)), None)

In [5]:
lyrics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362237 entries, 0 to 362236
Data columns (total 2 columns):
genre     362237 non-null object
lyrics    266557 non-null object
dtypes: object(2)
memory usage: 5.5+ MB


In [0]:
lyrics_df = lyrics_df[lyrics_df['lyrics'].notnull()]

In [0]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [0]:
def cleanText(text):
  text = text.lower()
  text = re.sub(r'\d', '#', text) 
  text = re.sub(r"\s+"," ", text, flags = re.I)
  tokens = tokenizer.tokenize(text.lower())
  tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation]
  
  return tokens if len(tokens) > 0 else None

In [0]:
lyrics_df['clean_lyrics'] = lyrics_df['lyrics'].apply(cleanText)

In [0]:
lyrics_df = lyrics_df[lyrics_df['clean_lyrics'].notnull()]

In [11]:
lyrics_df[['genre', 'clean_lyrics']].sample(3)

,genre,clean_lyrics
98211,Pop,"[black, colour, true, loves, hair, lips, like, roses, fair, sweetest, smile, gentlest, hands, love, ground, whereon, stands, love, love, well, knows, love, ground, whereon, goes, wish, day, would, soon, come, one, go, clyde, mourn, weep, satisfied, never, sleep, write, letters, short, lines, suffer, death, ten, thousand, times, black, colour, true, loves, hair, lips, like, roses, fair, sweetest, smile, gentlest, hands, love, ground, whereon, stands, love, ground, whereon, stands, love, love, love, ground, whereon, stands]"
14348,Rock,"[know, gotcha, figured, proud, walkin, tall, whach, whach, whach, make, know, give, take, blame, someone, else, everyone, except, one, step, forward, seems, get, sink, faster, know, want, save, day, grab, reins, run, circles, gonna, crash, drag, world, one, thing, remains, hey, man, aware, seem, day, dont, care, anyway, run, cant, hide, guilt, u, built, inside, forget, never, get, thinking, backwards, hell, get, started, drowning, faster, go, wanna, save, day, grab, reins, run, circles, gonna, crash, drag, world, one, thing, remains, wanna, save, day, grab, reins, run, circles, gonna, crash, drag, world, one, thing, remains]"
136701,Country,"[tell, wondering, think, youve, gone, mad, speaking, another, language, one, understands, talk, ocean, listen, birds, everyone, around, wonders, heard, youre, broken, youre, beautiful, fit, inside, box, may, seem, blurry, rest, thats, cause, see, colors, one, sees, youre, kaleidoscope, run, chase, sunrise, wish, falling, stars, paint, sky, eyes, hold, moon, arms, youre, broken, youre, beautiful, fit, inside, box, may, seem, blurry, rest, thats, cause, see, colors, one, sees, youre, kaleidoscope, youre, kaleidoscope, instrumental]"


In [0]:
FEATURE_VECTOR_SIZE = 300 
WINDOW_SIZE = 5
all_sentenses = lyrics_df["clean_lyrics"].tolist()
wevec_model = Word2Vec(all_sentenses, size=FEATURE_VECTOR_SIZE, window=WINDOW_SIZE, workers=4, min_count=3)

### Review most similar words
Get initial evaluation of the word vectors by analyzing the most similar words for a few interesting words in the text. 

Choose words yourself, and find the most similar words to them.

In [14]:
for word in ['love', 'man', 'girl', 'feel']:
  print('**********{}**************'.format(word))
  print(wevec_model.wv.most_similar('love'))
  print('**********{}**************'.format(word))
  

**********love**************
[('know', 0.6976536512374878), ('baby', 0.6739638447761536), ('cause', 0.6532072424888611), ('true', 0.620837926864624), ('oh', 0.6157742738723755), ('want', 0.6149493455886841), ('never', 0.6118445992469788), ('always', 0.6080036163330078), ('heart', 0.6009585857391357), ('way', 0.5991636514663696)]
**********love**************
**********man**************
[('know', 0.6976536512374878), ('baby', 0.6739638447761536), ('cause', 0.6532072424888611), ('true', 0.620837926864624), ('oh', 0.6157742738723755), ('want', 0.6149493455886841), ('never', 0.6118445992469788), ('always', 0.6080036163330078), ('heart', 0.6009585857391357), ('way', 0.5991636514663696)]
**********man**************
**********girl**************
[('know', 0.6976536512374878), ('baby', 0.6739638447761536), ('cause', 0.6532072424888611), ('true', 0.620837926864624), ('oh', 0.6157742738723755), ('want', 0.6149493455886841), ('never', 0.6118445992469788), ('always', 0.6080036163330078), ('heart', 0

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


### Word Vectors Algebra
We've seen in class examples of algebraic games on the word vectors (e.g. man - woman + king = queen ). 

Try a few vector algebra terms, and evaluate how well they work. Try to use the Cosine distance and compare it to the Euclidean distance.

In [15]:
v1 = wevec_model.wv['king'] 
v2 = wevec_model.wv['queen'] 

res =  v1 - v2 
wevec_model.wv.similar_by_vector(res)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('king', 0.4416906237602234),
 ('victorious', 0.28422659635543823),
 ('christ', 0.28422001004219055),
 ('overthrow', 0.2781832814216614),
 ('disarmament', 0.2606436610221863),
 ('undisputed', 0.2583610415458679),
 ('overcome', 0.25667399168014526),
 ('solomon', 0.2539622485637665),
 ('dispear', 0.24596869945526123),
 ('battle', 0.23669087886810303)]

In [17]:
v1 = wevec_model.wv['woman'] 
v2 = wevec_model.wv['girl'] 

res =  v1 - v2 
wevec_model.wv.similar_by_vector(res)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('woman', 0.3945504426956177),
 ('witchy', 0.3829672932624817),
 ('adherence', 0.3591882586479187),
 ('hath', 0.355106920003891),
 ('wastelands', 0.35092365741729736),
 ('hinguckt', 0.3403870761394501),
 ('wanhopige', 0.3390408158302307),
 ('incarnate', 0.33836591243743896),
 ('parachutist', 0.3364051580429077),
 ('dissatisfied', 0.3338039517402649)]

In [18]:
v1 = wevec_model.wv['girl'] 
v2 = wevec_model.wv['man'] 

res =  v1 + v2 
wevec_model.wv.similar_by_vector(res)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('girl', 0.8658943176269531),
 ('man', 0.8452440500259399),
 ('woman', 0.7247971892356873),
 ('boy', 0.7180936336517334),
 ('know', 0.6527124047279358),
 ('baby', 0.6476815938949585),
 ('cause', 0.638348400592804),
 ('got', 0.6022270321846008),
 ('yeah', 0.5943912267684937),
 ('love', 0.5738491415977478)]

In [19]:
v1 = wevec_model.wv['woman'] 
v2 = wevec_model.wv['husband'] 

res =  v1 + v2 
wevec_model.wv.similar_by_vector(res)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('woman', 0.8930797576904297),
 ('husband', 0.7790439128875732),
 ('wife', 0.6306020021438599),
 ('man', 0.6067594289779663),
 ('girl', 0.5493606328964233),
 ('daughter', 0.5384731292724609),
 ('child', 0.49661391973495483),
 ('women', 0.48693349957466125),
 ('neighbor', 0.46220311522483826),
 ('lovin', 0.4582667052745819)]

In [20]:
v1 = wevec_model.wv['wife'] 
v2 = wevec_model.wv['woman'] 

res =  v1 - v2 
wevec_model.wv.similar_by_vector(res)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('wife', 0.5940633416175842),
 ('doorjust', 0.39944425225257874),
 ('beaters', 0.38050615787506104),
 ('tillman', 0.3772699236869812),
 ('nephews', 0.3584328591823578),
 ('ahhhhaaaaaaaaghhhh', 0.3539944887161255),
 ('attended', 0.34795355796813965),
 ('swipe', 0.33600378036499023),
 ('saver', 0.3341713845729828),
 ('burglary', 0.32993119955062866)]

In [26]:
v1 = wevec_model.wv['young'] 
v2 = wevec_model.wv['baby'] 

res =  v1 + v2 
wevec_model.wv.similar_by_vector(res)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('young', 0.7975558042526245),
 ('baby', 0.7816184163093567),
 ('girl', 0.6593176126480103),
 ('love', 0.6478465795516968),
 ('cause', 0.6436699628829956),
 ('know', 0.6418748497962952),
 ('yeah', 0.6136243343353271),
 ('boy', 0.6044150590896606),
 ('got', 0.5908496975898743),
 ('want', 0.58655846118927)]

## Sentiment Analysis
Estimate sentiment of words using word vectors.  
In this section, we'll use the SemEval-2015 English Twitter Sentiment Lexicon.  
The lexicon was used as an official test set in the SemEval-2015 shared Task #10: Subtask E, and contains a polarity score for words in range -1 (negative) to 1 (positive) - http://saifmohammad.com/WebPages/SCL.html#OPP

Build a classifier for the sentiment of a word given its word vector. Split the data to a train and test sets, and report the model performance on both sets.

In [27]:
twitter_lex_df = pd.read_csv('/content/drive/My Drive/SemEval2015-English-Twitter-Lexicon.txt', sep='\t',  names =['polarity', 'word'])
twitter_lex_df.describe(include = 'all')

,polarity,word
count,1515.000000,1515
unique,NaN,1515
top,NaN,giveaway
freq,NaN,1
mean,0.000319,NaN
std,0.502247,NaN
min,-0.984000,NaN
25%,-0.422000,NaN
50%,0.031000,NaN
75%,0.406000,NaN


In [28]:
twitter_lex_df.sample(3)

,polarity,word
382,0.391,no rush
584,0.188,read
390,0.391,proper


In [0]:
twitter_lex_df['vector'] = twitter_lex_df["word"].apply(lambda word: wevec_model.wv[word].reshape(1,-1) if word in wevec_model.wv else None)
twitter_lex_df = twitter_lex_df[twitter_lex_df.vector.notnull()]


In [0]:
  
train, test = train_test_split(twitter_lex_df, test_size=0.2, random_state=999)
X_train = train['vector'].values
y_train = train['polarity'].values
X_test = test['vector'].values
y_test = test['polarity'].values

X_train = np.concatenate(X_train).reshape(X_train.shape[0], X_train[0].shape[1])

shape = list(X_test[0].shape)
X_test = np.concatenate(X_test).reshape(X_test.shape[0],X_test[0].shape[1])

In [35]:
model = XGBRegressor()
model.fit(X_train, y_train)

[15:24:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [36]:
model.score(X_test, y_test)

0.35175120519967285

In [41]:
use_cuda = torch.cuda.is_available()
use_cuda

True

In [0]:
import torch.utils.data as utils_data
from torch.autograd import Variable

training_samples = utils_data.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
data_loader = utils_data.DataLoader(training_samples, batch_size=5, shuffle=False)	

test_samples = utils_data.TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
test_data_loader = utils_data.DataLoader(test_samples, batch_size=len(X_test), shuffle=False)	
test_data_loader_1 = utils_data.DataLoader(test_samples, batch_size=1, shuffle=False)


In [0]:
import torch
from torch import nn, optim
from torch.nn import functional as F

class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)
        self.predict = torch.nn.Linear(n_hidden, n_output)
        self.dropout1 = nn.Dropout(p=0.1)
        

    def forward(self, x):
        x = F.relu(self.hidden(x))   
        x = self.dropout1(x)
        x = self.predict(x)  
        return x

      
      
net = Net(n_feature=300, n_hidden=20, n_output=1)     
optimizer = torch.optim.SGD(net.parameters(), lr=0.001)
loss_func = torch.nn.MSELoss()

if use_cuda:
  net = net.cuda()
  
# plt.ion()  

for epoch in range(1000):
  
  batchloss = 0.0
  for batch_idx, (data, target) in enumerate(data_loader):
        
    if use_cuda:
      data = data.cuda()
      target = target.cuda()
          
          
    data, target = Variable(data).float(), Variable(target).float()
    optimizer.zero_grad()
    prediction = net(data)
    loss = loss_func(prediction, target) 
        
    loss.backward() 
    optimizer.step()
    
    batchloss += loss.item()
  
  print(batchloss)
        

def score(model, data_loader, criterion):
  
  running_loss = 0.
  calc_count =0.
  
  model.eval()
  
  with torch.no_grad():
    for images, labels in data_loader:
      if use_cuda:
        images = images.cuda()
        labels = labels.cuda()
        
      images = Variable(images)  
      labels = Variable(labels)
      
      outputs = model(images.float())
      loss = criterion(outputs, labels.float())
      return loss 

score(net, test_data_loader, loss_func ).item()  

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


35.32020165209542
34.12065568752587
33.36858466826379
32.861710192635655
32.46283303387463
32.039268956985325
31.791825458873063
31.47485022060573
31.27802703413181
30.98533385974588
30.876199471298605
30.97434275317937
30.450614749686792
30.566897857235745
30.258383152307943
30.269483193871565
30.196752968011424
30.2268483436128
30.121287770103663
29.820330894202925
29.84012677008286
29.766415531401435
29.723762664943933
29.81244556279853
29.72858191933483
29.68568113958463
29.509966120764147
29.457513962493977
29.521088231587783
29.58354257029714
29.415298566178535
29.479346899315715
29.40049005218316
29.37178146687802
29.313269121154008
29.359418812392505
29.20186443568673
29.282623132690787
29.193385927175996
29.184797887261993
29.193913496957975
29.18273541907547
29.03546228356754
28.985123107981963
29.01842256053351
29.123007585254527
29.02714754010958
28.907948882551864
29.068279995030025
28.853292530773615
28.87027514021611
28.961691731587052
28.832644348680333
28.9887793098314

Use your trained model from the previous question to predict the sentiment score of words in the lyrics corpus that are not part of the original sentiment dataset. Review the words with the highest positive and negative sentiment. Do the results make sense?

### Visualize Word Vectors
In this section, you'll plot words on a 2D grid based on their inner similarity. We'll use the tSNE transformation to reduce dimensions from 300 to 2. You can get sample code from https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial or other tutorials online.

Perform the following:
- Keep only the 3,000 most frequent words (after removing stopwords)
- For this list, compute for each word its relative abundance in each of the genres
- Compute the ratio between the proportion of each word in each genre and the proportion of the word in the entire corpus (the background distribution)
- Pick the top 50 words for each genre. These words give good indication for that genre. Join the words from all genres into a single list of top significant words. 
- Compute tSNE transformation to 2D for all words, based on their word vectors
- Plot the list of the top significant words in 2D. Next to each word output its text. The color of each point should indicate the genre for which it is most significant.

You might prefer to use a different number of points or a slightly different methodology for improved results.  
Analyze the results.

## Text Classification
In this section, you'll build a text classifier, determining the genre of a song based on its lyrics.

### Text classification using Bag-of-Words
Build a Naive Bayes classifier based on the bag of Words.  
You will need to divide your dataset into a train and test sets.

Show the confusion matrix.

Show the classification report - precision, recall, f1 for each class.

### Text classification using Word Vectors
#### Average word vectors
Do the same, using a classifier that averages the word vectors of words in the document.

#### TfIdf Weighting
Do the same, using a classifier that averages the word vectors of words in the document, weighting each word by its TfIdf.


### Text classification using ConvNet
Do the same, using a ConvNet.  
The ConvNet should get as input a 2D matrix where each column is an embedding vector of a single word, and words are in order. Use zero padding so that all matrices have a similar length.  
Some songs might be very long. Trim them so you keep a maximum of 128 words (after cleaning stop words and rare words).  
Initialize the embedding layer using the word vectors that you've trained before, but allow them to change during training.  

Extra: Try training the ConvNet with 2 slight modifications:
1. freezing the the weights trained using Word2vec (preventing it from updating)
1. random initialization of the embedding layer

You are encouraged to try this question on your own.  

You might prefer to get ideas from the paper "Convolutional Neural Networks for Sentence Classification" (Kim 2014, [link](https://arxiv.org/abs/1408.5882)).

There are several implementations of the paper code in PyTorch online (see for example [this repo](https://github.com/prakashpandey9/Text-Classification-Pytorch) for a PyTorch implementation of CNN and other architectures for text classification). If you get stuck, they might provide you with a reference for your own code.